In [11]:
import pandas as pd

data = pd.read_csv('games_original.txt', delimiter=" ### ")

<ipython-input-11-25c5d53bc817>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv('games_original.txt', delimiter=" ### ")


,# 1.t 2.date 3.result 4.welo 5.belo 6.len 7.date_c 8.resu_c 9.welo_c 10.belo_c 11.edate_c 12.setup 13.fen 14.resu2_c 15.oyrange 16.bad_len,17.game...
0,1 2000.03.14 1-0 2851 None 67 date_false resul...,W1.d4 B1.d5 W2.c4 B2.e6 W3.Nc3 B3.Nf6 W4.cxd5 ...
1,2 2000.03.14 1-0 2851 None 53 date_false resul...,W1.e4 B1.d5 W2.exd5 B2.Qxd5 W3.Nc3 B3.Qa5 W4.d...
2,3 1999.11.20 1-0 2851 None 57 date_false resul...,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bc4 B3.Bc5 W4.c3 ...
3,4 1999.11.20 1-0 2851 None 49 date_false resul...,W1.e4 B1.d5 W2.exd5 B2.Qxd5 W3.Nc3 B3.Qa5 W4.d...
4,5 2000.02.20 1/2-1/2 2851 2633 97 date_false r...,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bb5 B3.a6 W4.Ba4 ...
...,...,...
3561465,3561466 1967.??.?? 1-0 None None 57 date_true ...,W1.e4 B1.e6 W2.d4 B2.d5 W3.e5 B3.c5 W4.dxc5 B4...
3561466,3561467 1997.??.?? 1/2-1/2 None None 80 date_t...,W1.d4 B1.Nf6 W2.Nc3 B2.g6 W3.Bg5 B3.Bg7 W4.Nf3...
3561467,3561468 1961.01.03 1-0 None None 57 date_false...,W1.c4 B1.Nf6 W2.Nc3 B2.g6 W3.e4 B3.d6 W4.d4 B4...
3561468,3561469 1997.??.?? 1-0 None None 80 date_true ...,W1.g3 B1.d5 W2.Bg2 B2.c5 W3.d3 B3.Nf6 W4.Nf3 B...


In [12]:
data[data.columns[0].split(" ")] = data["# 1.t 2.date 3.result 4.welo 5.belo 6.len 7.date_c 8.resu_c 9.welo_c 10.belo_c 11.edate_c 12.setup 13.fen 14.resu2_c 15.oyrange 16.bad_len"].str.split(expand=True)

In [17]:
data = data.drop(labels=data.columns[0], axis=1)

In [21]:
data.head(5)

,17.game...,#,1.t,2.date,3.result,4.welo,5.belo,6.len,7.date_c,8.resu_c,9.welo_c,10.belo_c,11.edate_c,12.setup,13.fen,14.resu2_c,15.oyrange,16.bad_len
0,W1.d4 B1.d5 W2.c4 B2.e6 W3.Nc3 B3.Nf6 W4.cxd5 ...,1,2000.03.14,1-0,2851,None,67,date_false,result_false,welo_false,belo_true,edate_true,setup_false,fen_false,result2_false,oyrange_false,blen_false,None
1,W1.e4 B1.d5 W2.exd5 B2.Qxd5 W3.Nc3 B3.Qa5 W4.d...,2,2000.03.14,1-0,2851,None,53,date_false,result_false,welo_false,belo_true,edate_true,setup_false,fen_false,result2_false,oyrange_false,blen_false,None
2,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bc4 B3.Bc5 W4.c3 ...,3,1999.11.20,1-0,2851,None,57,date_false,result_false,welo_false,belo_true,edate_false,setup_false,fen_false,result2_false,oyrange_false,blen_false,None
3,W1.e4 B1.d5 W2.exd5 B2.Qxd5 W3.Nc3 B3.Qa5 W4.d...,4,1999.11.20,1-0,2851,None,49,date_false,result_false,welo_false,belo_true,edate_false,setup_false,fen_false,result2_false,oyrange_false,blen_false,None
4,W1.e4 B1.e5 W2.Nf3 B2.Nc6 W3.Bb5 B3.a6 W4.Ba4 ...,5,2000.02.20,1/2-1/2,2851,2633,97,date_false,result_false,welo_false,belo_false,edate_false,setup_false,fen_false,result2_false,oyrange_false,blen_false,None
